---
title: pyOpenSci Current Software Review Stats
subtitle: pyOpenSci Peer Review Summary Stats
author:
  - name: Leah Wasser
    affiliations: pyOpenSci
    orcid: 0000-0002-7859-8394
    email: leah@pyopensci.org
license:
  code: MIT
date: 2024/06/20
---


* https://github.com/ryantam626/jupyterlab_code_formatter

This is a workflow that colates all GitHub issues associated with our reviews. 

Questions i have

* How to add figure captions and alt text
* 

In [1]:
from datetime import datetime, timezone
import warnings
import pytz

import altair as alt
import pandas as pd

from pyosmeta import ProcessIssues
from pyosmeta.github_api import GitHubAPI

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
# Open Reviews

In [3]:
# Get a list of reviews submitted to us
# This potentially doesn't include issues that were deemed out of scope...
github_api = GitHubAPI(
    org="pyopensci",
    repo="software-submission",
    labels=[
        "0/seeking-editor",
        "1/editor-assigned",
        "2/seeking-reviewers",
        "3/reviewers-assigned",
        "4/reviews-in-awaiting-changes",
        "5/awaiting-reviewer-response",
        "7/under-joss-review",
        "8/joss-review-complete",
        "New Submission!",
    ],
)
process_review = ProcessIssues(github_api)
issues = process_review.get_issues()
reviews, errors = process_review.parse_issues(issues)
review_table = [{"package_name": name,
                "created_at": review.created_at,
                "date_closed": review.closed_at,
                "editor": review.editor.github_username,
                #"editor": review.editor.name,
                "labels": review.labels} 
                 for name, review in reviews.items()]


## Current open reviews & total days open


In [4]:
reviews_df = pd.DataFrame(review_table)
open_reviews = reviews_df[reviews_df['date_closed'].isna()]

today = datetime.now(timezone.utc)
open_reviews['days_open'] = (today - open_reviews['created_at']).dt.days
open_reviews.drop(columns=['date_closed'], inplace=True)
open_reviews['created_at'] = open_reviews['created_at'].dt.date
open_reviews

,package_name,created_at,editor,labels,days_open
0,MontePy,2024-07-01,TBD,"[0/pre-review-checks, New Submission!]",11
1,Great Tables,2024-06-14,TBD,[0/seeking-editor],28
2,Stingray,2024-06-14,hamogu,"[2/seeking-reviewers, astropy]",28
3,Fluidimage,2024-05-30,TBD,[0/seeking-editor],43
4,astrodata,2024-05-13,hamogu,"[3/reviewers-assigned, astropy]",60
5,QuadratiK,2024-05-13,isabelizimm,[1/editor-assigned],60
6,PyPartMC,2024-05-03,TBD,[0/seeking-editor],70
7,ANDES,2024-04-22,TBD,"[0/pre-review-checks, New Submission!, on-hold]",81
8,CyNetDiff,2024-04-22,sneakers-the-rat,[1/editor-assigned],81
9,AMS,2024-04-22,TBD,"[2/seeking-reviewers, 1/editor-assigned]",82


In [11]:
# Get Presubmission inquiries
gh_presubmissions = GitHubAPI(
    org="pyopensci", repo="software-submission", labels=["presubmission"]
)
process_review = ProcessIssues(gh_presubmissions)
pre_issues = process_review.get_issues()
pre_submissions, errors = process_review.parse_issues(pre_issues)
pre_submission_table = [
    {"package_name": name,
        "created_at": review.created_at,
        "date_closed": review.closed_at,
        "labels": review.labels,
    }
    for name, review in pre_submissions.items()
]

## All presubmissions

In [10]:
presubmission_df = pd.DataFrame(pre_submission_table)

#today = datetime.now(timezone.utc)
#open_reviews['days_open'] = (today - open_reviews['created_at']).dt.days
#open_reviews.drop(columns=['date_closed'], inplace=True)
#open_reviews['created_at'] = open_reviews['created_at'].dt.date
# ToDo calculate days open
all_presubmissions = len(presubmission_df)
all_presubmissions

52

In [14]:
# Get all currently open presubmissions 
open_presubmissions = presubmission_df[presubmission_df['date_closed'].isna()]
total_open = len(open_presubmissions)

## Currently open software presubmission inquiries

* It could be useful to grab the most recent comments on each
* It would also be useful to grab the gh usernames of all people involved in the discussion and credit them. So for one i see astropy editors + alex being involved.

There are **{eval}`total_open` presubmission requests** currently open. 

## Current open softwrae review submissions 

Next we explore the currently open issues.

pyOpenSci currently has **{eval}`total_open`** total open submissions.

* x of these are in active review
* x of these are in pre-review
* x of these are being submitted to JOSS
* x of these have been approvved and are a part of our ecosystem 

In [ ]:
# TODO: add editor for each review? 
# TODO: list all current editors 

In [ ]:
# Get all currently closed / approved issues 

# calculate time that they were in review. 